In [2]:
!pip install selenium

In [3]:
## 필요한 패키지들 import
# 셀레니움 크롤링 시 필요한 패키지
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests as req
from selenium import webdriver as wb
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

# 이벤트 제어 라이브러리 (클릭, 드래그 앤 드롭 등등)
from selenium.webdriver.common.action_chains import ActionChains

# 크롤링 간 대기시간을 설정해, 오류를 방지하기 위함.
import time

In [3]:
# 가상 브라우저 실행 및 사이트 띄우기
driver = wb.Chrome()
driver.get("https://nfds.go.kr/stat/general.do")

# 4.29일자 추가 크롤링
- 2017년 데이터 추가 크롤링
- 일별 데이터 > 월별 데이터로 바꾸기

In [56]:
# 반복문 전 작업

# 검색 전, 검색 칸의 내용을 지워주는 코드.
date_start = driver.find_element(By.NAME, "datePicker1s")
date_start.clear()

date_end = driver.find_element(By.NAME, "datePicker1e")
date_end.clear()

# 검색하기 누르는 코드
# 객체 생성
actions = ActionChains(driver)

# 검색하기 위치 지정, 검색하기 : elm_click.click()
# searchItem > div.search-words > div.search-option > div.page-search > a:nth-child(1) > button 
elm_click = driver.find_element(By.CSS_SELECTOR,
                                "#searchItem > div.search-words > div.search-option > div.page-search > a:nth-child(1) > button")

# 지역구 리스트부터 만든다.
region_list = ["강남구", "강동구", "강북구", "강서구", "관악구", "광진구",
               "구로구", "금천구", "노원구", "도봉구", "동대문구", "동작구",
              "마포구", "서대문구", "서초구", "성동구", "성북구", "송파구",
              "양천구", "영등포구", "용산구", "은평구", "종로구", "중구", "중랑구"]

# 날짜 데이터를 2017년도로 한정짓는다.
date = pd.date_range(start='20170101', end='20171231')

date_list = date.strftime("%Y-%m-%d").tolist()

NameError: name 'driver' is not defined

In [8]:
# 원하는 정보를 크롤링해오는 코드
# 2017년 데이터가 필요해서, 추가로 크롤링해 온다.

# 화재 발생 건수를 담을 빈 리스트를 만든다
fire_counts = []

# 반복문 시작
for date in date_list :
    # 검색칸 중 시작날짜에 있는 내용을 지워준다.
    element_start = driver.find_element(By.NAME, "datePicker1s")
    element_start.clear()

    # 종료날짜도 똑같이 지워준다.
    element_end = driver.find_element(By.NAME, "datePicker1e")
    element_end.clear()
    
    # 시작날짜, 종료날짜에 처음부터 끝까지의 날짜 넣기
    date_start.send_keys(date) ; date_end.send_keys(date)
    
    # 검색 실행 > 데이터가 나온다.
    elm_click.click()
    
    # 클릭 이후 기다리면서 에러 방지하기
    # 기다려 주지 않으면, 날짜가 바뀌면서 정보가 바뀌는데, 그 속도가 컴퓨터의 속도보다 한참 느려 에러가 발생한다.
    time.sleep(1)
    
    # 이제 지역구 별로 화재발생 건수를 크롤링해온다.
    # f-string을 이용하여 선택자의 내부 숫자만 바꿀 예정인데, 이를 위해 선택자의 수를 파악할 필요가 있다.
    
    # 강남구 화재건수 : #grdViewer > table > tbody > tr:nth-child(4) > td:nth-child(2)
    # 중랑구 화재건수 : #grdViewer > table > tbody > tr:nth-child(28) > td:nth-child(2)
    # 이 상황에서, 4에서 28까지의 숫자를 고려하여 for 반복문을 설정해 준다.
    for i in range(4, 28 + 1) : # 4 ~ 28 = 강남구 ~ 중랑구
        fire_count = int(driver.find_element(By.CSS_SELECTOR, 
                                             f"#grdViewer > table > tbody > tr:nth-child({i}) > td:nth-child(2)").text)
        fire_counts.append(fire_count)
    

In [10]:
# 위 크롤링 정보를 바탕으로 데이터프레임 생성
# 화재건수 데이터의 형태 변경
import numpy as np

# 열에는 지역구 이름 쓸것이므로, region_list에 맞게 재배열
fire_counts = np.array(fire_counts).reshape(-1, len(region_list))

df = pd.DataFrame(fire_counts, columns=region_list, index=date_list) ; df

강남구  강동구  강북구  강서구  관악구  광진구  구로구  금천구  노원구  도봉구  ...  성동구  성북구  \
2017-01-01    1    2    1    2    1    0    0    0    0    0  ...    0    0   
2017-01-02    2    0    0    1    0    1    0    0    1    0  ...    0    0   
2017-01-03    3    0    1    3    1    0    0    0    0    0  ...    0    0   
2017-01-04    1    0    1    1    1    0    1    1    0    0  ...    0    2   
2017-01-05    1    1    0    0    0    0    0    0    0    0  ...    1    0   
...         ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
2017-12-27    0    2    0    0    1    0    0    1    0    0  ...    1    1   
2017-12-28    0    2    0    0    2    1    0    0    6    0  ...    0    0   
2017-12-29    1    0    0    0    2    0    1    0    1    1  ...    2    2   
2017-12-30    0    0    1    0    2    0    0    0    2    1  ...    0    1   
2017-12-31    0    0    2    0    0    0    0    0    1    1  ...    1    0   

            송파구  양천구  영등포구  용산구  은평구  종로구  중구  중랑구  
2017-01-01    2    1     0    0    0    0   0    0  
2017-01-02    0    0     1    0    1    0   0    1  
2017-01-03    1    0     0    0    0    0   0    0  
2017-01-04    1    0     0    0    0    0   0    0  
2017-01-05    0    0     0    1    1    1   0    0  
...         ...  ...   ...  ...  ...  ...  ..  ...  
2017-12-27    1    0     1    1    0    0   0    0  
2017-12-28    0    0     0    1    0    0   1    0  
2017-12-29    0    0     0    0    1    1   1    2  
2017-12-30    0    0     1    0    0    1   1    0  
2017-12-31    2    1     0    0    0    1   0    0  

[365 rows x 25 columns]

In [11]:
# 결과값 csv 파일로 저장
# 디폴트값인 utf-8로 저장시 파일 깨짐 > 한글 인코딩 방식인 cp949 사용 (euc-kr도 깨짐)
df.to_csv("2017년 화재발생건수.csv", encoding='cp949')

# 데이터 전처리
- 날짜 컬럼의 컬럼명 변경
- 각 구별로 작은 데이터프레임으로 나누기

In [64]:
# csv 파일을 다시 불러오기
df = pd.read_csv("2017년 화재발생건수.csv", encoding = 'cp949') ; df

Unnamed: 0  강남구  강동구  강북구  강서구  관악구  광진구  구로구  금천구  노원구  ...  성동구  성북구  \
0    2017-01-01    1    2    1    2    1    0    0    0    0  ...    0    0   
1    2017-01-02    2    0    0    1    0    1    0    0    1  ...    0    0   
2    2017-01-03    3    0    1    3    1    0    0    0    0  ...    0    0   
3    2017-01-04    1    0    1    1    1    0    1    1    0  ...    0    2   
4    2017-01-05    1    1    0    0    0    0    0    0    0  ...    1    0   
..          ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
360  2017-12-27    0    2    0    0    1    0    0    1    0  ...    1    1   
361  2017-12-28    0    2    0    0    2    1    0    0    6  ...    0    0   
362  2017-12-29    1    0    0    0    2    0    1    0    1  ...    2    2   
363  2017-12-30    0    0    1    0    2    0    0    0    2  ...    0    1   
364  2017-12-31    0    0    2    0    0    0    0    0    1  ...    1    0   

     송파구  양천구  영등포구  용산구  은평구  종로구  중구  중랑구  
0      2    1     0    0    0    0   0    0  
1      0    0     1    0    1    0   0    1  
2      1    0     0    0    0    0   0    0  
3      1    0     0    0    0    0   0    0  
4      0    0     0    1    1    1   0    0  
..   ...  ...   ...  ...  ...  ...  ..  ...  
360    1    0     1    1    0    0   0    0  
361    0    0     0    1    0    0   1    0  
362    0    0     0    0    1    1   1    2  
363    0    0     1    0    0    1   1    0  
364    2    1     0    0    0    1   0    0  

[365 rows x 26 columns]

In [65]:
# 날짜 컬럼의 컬럼명을 day로 변경
df.rename(columns = {"Unnamed: 0" : "day"}, inplace = True) ; df

# df의 인덱스를 day로 설정하고, 기존에 있던 day라는 컬럼 제거
df = df.set_index(keys = df["day"])
df = df.drop("day", axis = 1) ; df

강남구  강동구  강북구  강서구  관악구  광진구  구로구  금천구  노원구  도봉구  ...  성동구  성북구  \
day                                                           ...             
2017-01-01    1    2    1    2    1    0    0    0    0    0  ...    0    0   
2017-01-02    2    0    0    1    0    1    0    0    1    0  ...    0    0   
2017-01-03    3    0    1    3    1    0    0    0    0    0  ...    0    0   
2017-01-04    1    0    1    1    1    0    1    1    0    0  ...    0    2   
2017-01-05    1    1    0    0    0    0    0    0    0    0  ...    1    0   
...         ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
2017-12-27    0    2    0    0    1    0    0    1    0    0  ...    1    1   
2017-12-28    0    2    0    0    2    1    0    0    6    0  ...    0    0   
2017-12-29    1    0    0    0    2    0    1    0    1    1  ...    2    2   
2017-12-30    0    0    1    0    2    0    0    0    2    1  ...    0    1   
2017-12-31    0    0    2    0    0    0    0    0    1    1  ...    1    0   

            송파구  양천구  영등포구  용산구  은평구  종로구  중구  중랑구  
day                                                 
2017-01-01    2    1     0    0    0    0   0    0  
2017-01-02    0    0     1    0    1    0   0    1  
2017-01-03    1    0     0    0    0    0   0    0  
2017-01-04    1    0     0    0    0    0   0    0  
2017-01-05    0    0     0    1    1    1   0    0  
...         ...  ...   ...  ...  ...  ...  ..  ...  
2017-12-27    1    0     1    1    0    0   0    0  
2017-12-28    0    0     0    1    0    0   1    0  
2017-12-29    0    0     0    0    1    1   1    2  
2017-12-30    0    0     1    0    0    1   1    0  
2017-12-31    2    1     0    0    0    1   0    0  

[365 rows x 25 columns]

In [67]:
# 지역구 리스트부터 만든다.
region_list = ["강남구", "강동구", "강북구", "강서구", "관악구", "광진구",
               "구로구", "금천구", "노원구", "도봉구", "동대문구", "동작구",
              "마포구", "서대문구", "서초구", "성동구", "성북구", "송파구",
              "양천구", "영등포구", "용산구", "은평구", "종로구", "중구", "중랑구"]

# 반복문을 통해 각 구별 csv를 다운받는다.
for region in region_list :
    df.rename(columns = {f"{region}" : "발생 건수"})[["발생 건수"]].to_csv(f"2017년 {region} 화재발생건수.csv", encoding = "cp949")
    
# 결과 : 2017년 강남구 화재발생건수 ~ 2017년 중랑구 화재발생건수.csv

In [68]:
# 결과 예시 : 강남구
pd.read_csv("2017년 강남구 화재발생건수.csv", encoding = 'cp949')

day  발생 건수
0    2017-01-01      1
1    2017-01-02      2
2    2017-01-03      3
3    2017-01-04      1
4    2017-01-05      1
..          ...    ...
360  2017-12-27      0
361  2017-12-28      0
362  2017-12-29      1
363  2017-12-30      0
364  2017-12-31      0

[365 rows x 2 columns]

## 기존 데이터(2018~2022)와 2017년 데이터 합치기

In [72]:
# 강남구 데이터로 테스트
# 각 구에 대한 두 개의 CSV 파일 로드
df1 = pd.read_csv('2017년 강남구 화재발생건수.csv', encoding = 'cp949')
df2 = pd.read_csv('강남구 화재발생건수.csv', encoding = 'cp949')

# 두 개의 데이터프레임을 합치기 ()
df = pd.concat([df1, df2], axis=0)

# CSV 파일로 저장
df.to_csv('강남구 전체 화재발생건수.csv', index=False, encoding = "cp949") ; df

day  발생 건수
0     2017-01-01      1
1     2017-01-02      2
2     2017-01-03      3
3     2017-01-04      1
4     2017-01-05      1
...          ...    ...
1821  2022-12-27      1
1822  2022-12-28      2
1823  2022-12-29      1
1824  2022-12-30      2
1825  2022-12-31      1

[2191 rows x 2 columns]

In [73]:
import pandas as pd

for region in region_list:
    # 각 구에 대한 두 개의 CSV 파일 로드
    df1 = pd.read_csv(f'2017년 {region} 화재발생건수.csv', encoding = 'cp949')
    df2 = pd.read_csv(f'{region} 화재발생건수.csv', encoding = 'cp949')

    # 두 개의 데이터프레임을 합치기 ()
    df = pd.concat([df1, df2], axis=0, ignore_index=True)

    # csv 파일로 저장
    df.to_csv(f'{region} 전체 화재발생건수.csv', index=False, encoding = 'cp949')

In [74]:
# 예시
pd.read_csv("강남구 전체 화재발생건수.csv", encoding = 'cp949')

day  발생 건수
0     2017-01-01      1
1     2017-01-02      2
2     2017-01-03      3
3     2017-01-04      1
4     2017-01-05      1
...          ...    ...
2186  2022-12-27      1
2187  2022-12-28      2
2188  2022-12-29      1
2189  2022-12-30      2
2190  2022-12-31      1

[2191 rows x 2 columns]

## 일별 데이터 > 월별 데이터로 바꾸기

In [90]:
# 예시 : 강남구 데이터
df = pd.read_csv("강남구 전체 화재발생건수.csv", encoding = 'cp949') ; df

day  발생 건수
0     2017-01-01      1
1     2017-01-02      2
2     2017-01-03      3
3     2017-01-04      1
4     2017-01-05      1
...          ...    ...
2186  2022-12-27      1
2187  2022-12-28      2
2188  2022-12-29      1
2189  2022-12-30      2
2190  2022-12-31      1

[2191 rows x 2 columns]

In [91]:
# 'day' 열을 datetime 형식으로 변환
df['day'] = pd.to_datetime(df['day'])

# 'day' 열을 기준으로 그룹화하여 월별 합계 구하기 (day가 인덱스로 감)
df_monthly = df.resample('M', on='day').sum()

# 인덱스 부분의 표기를 변환
df_monthly.index = df_monthly.index.strftime('%Y-%m') ; df_monthly

발생 건수
day           
2017-01     56
2017-02     37
2017-03     41
2017-04     46
2017-05     53
...        ...
2022-08     37
2022-09     42
2022-10     34
2022-11     22
2022-12     53

[72 rows x 1 columns]

In [92]:
# csv 파일로 저장 (인덱스가 day라 삭제하면 안됨.)
df_monthly.to_csv('강남구 전체 월별 화재발생건수.csv', encoding = 'cp949')

In [93]:
# 이 과정을 반복문으로 표현한다.

for region in region_list:
    # csv 파일
    df = pd.read_csv(f'{region} 전체 화재발생건수.csv', encoding='cp949')

    # 'day' 열을 datetime 형식으로 변환
    df['day'] = pd.to_datetime(df['day'])

    # 'day' 열을 기준으로 그룹화하여 월별 합계 구하기 (day가 인덱스로 감)
    df_monthly = df.resample('M', on='day').sum()

    # 인덱스 부분의 표기를 변환
    df_monthly.index = df_monthly.index.strftime('%Y-%m')

    # csv 파일로 저장
    df_monthly.to_csv(f'{region} 전체 월별 화재발생건수.csv', encoding='cp949')

In [94]:
# 예시 
pd.read_csv("강남구 전체 월별 화재발생건수.csv", encoding = 'cp949')

day  발생 건수
0   2017-01     56
1   2017-02     37
2   2017-03     41
3   2017-04     46
4   2017-05     53
..      ...    ...
67  2022-08     37
68  2022-09     42
69  2022-10     34
70  2022-11     22
71  2022-12     53

[72 rows x 2 columns]

In [95]:
# 2022년 데이터를 빼야 한다. 예시는 강남구
df = pd.read_csv('강남구 전체 월별 화재발생건수.csv', encoding='cp949') ; df

df[df['day'] <= '2021-12-31'] 

day  발생 건수
0   2017-01     56
1   2017-02     37
2   2017-03     41
3   2017-04     46
4   2017-05     53
5   2017-06     37
6   2017-07     50
7   2017-08     33
8   2017-09     41
9   2017-10     33
10  2017-11     39
11  2017-12     36
12  2018-01     54
13  2018-02     47
14  2018-03     35
15  2018-04     42
16  2018-05     34
17  2018-06     30
18  2018-07     34
19  2018-08     47
20  2018-09     25
21  2018-10     26
22  2018-11     29
23  2018-12     33
24  2019-01     52
25  2019-02     33
26  2019-03     44
27  2019-04     42
28  2019-05     38
29  2019-06     48
30  2019-07     46
31  2019-08     31
32  2019-09     28
33  2019-10     30
34  2019-11     29
35  2019-12     35
36  2020-01     28
37  2020-02     19
38  2020-03     31
39  2020-04     35
40  2020-05     22
41  2020-06     34
42  2020-07     34
43  2020-08     30
44  2020-09     35
45  2020-10     34
46  2020-11     32
47  2020-12     53
48  2021-01     43
49  2021-02     35
50  2021-03     39
51  2021-04     38
52  2021-05     33
53  2021-06     30
54  2021-07     29
55  2021-08     32
56  2021-09     21
57  2021-10     28
58  2021-11     29
59  2021-12     34

In [96]:
# 예시 만들어서 보기 : 강남구
df.to_csv('21년도까지 강남 전체 월별 화재발생건수.csv', encoding = 'cp949', index = False)

In [97]:
pd.read_csv('21년도까지 강남 전체 월별 화재발생건수.csv', encoding = 'cp949')

day  발생 건수
0   2017-01     56
1   2017-02     37
2   2017-03     41
3   2017-04     46
4   2017-05     53
..      ...    ...
67  2022-08     37
68  2022-09     42
69  2022-10     34
70  2022-11     22
71  2022-12     53

[72 rows x 2 columns]

In [98]:
# 위 방식과 동일하게 재 지정한 후 csv 파일로 그대로 저장해준다.

for region in region_list:
    # csv 파일
    df = pd.read_csv(f'{region} 전체 월별 화재발생건수.csv', encoding='cp949')

    # 날짜 지정후 df 재저장
    df = df[df['day'] <= '2021-12-31']

    # csv 파일로 저장
    df.to_csv(f'{region} 전체 월별 화재발생건수.csv', encoding='cp949', index = False)

In [8]:
# 예시 확인
pd.read_csv("강남구 전체 월별 화재발생건수.csv", encoding = 'cp949')

day  발생 건수
0   2017-01     56
1   2017-02     37
2   2017-03     41
3   2017-04     46
4   2017-05     53
5   2017-06     37
6   2017-07     50
7   2017-08     33
8   2017-09     41
9   2017-10     33
10  2017-11     39
11  2017-12     36
12  2018-01     54
13  2018-02     47
14  2018-03     35
15  2018-04     42
16  2018-05     34
17  2018-06     30
18  2018-07     34
19  2018-08     47
20  2018-09     25
21  2018-10     26
22  2018-11     29
23  2018-12     33
24  2019-01     52
25  2019-02     33
26  2019-03     44
27  2019-04     42
28  2019-05     38
29  2019-06     48
30  2019-07     46
31  2019-08     31
32  2019-09     28
33  2019-10     30
34  2019-11     29
35  2019-12     35
36  2020-01     28
37  2020-02     19
38  2020-03     31
39  2020-04     35
40  2020-05     22
41  2020-06     34
42  2020-07     34
43  2020-08     30
44  2020-09     35
45  2020-10     34
46  2020-11     32
47  2020-12     53
48  2021-01     43
49  2021-02     35
50  2021-03     39
51  2021-04     38
52  2021-05     33
53  2021-06     30
54  2021-07     29
55  2021-08     32
56  2021-09     21
57  2021-10     28
58  2021-11     29
59  2021-12     34

## 날씨 데이터와 화재발생건수 합치기
- 독립변수 + 종속변수

In [10]:
# 날씨 데이터 import (cp949는 에러)
weather = pd.read_csv("날씨 전체 데이터.csv", encoding = "utf-8")

In [11]:
weather

Unnamed: 0   지점 지점명       일시  평균기온(°C)  최고기온(°C)  최저기온(°C)  평균풍속(m/s)  \
0              0  400  강남  2017-01     -0.40     11.60    -10.10        1.7   
1              1  400  강남  2017-02      1.10     12.10     -7.80        1.9   
2              2  400  강남  2017-03      7.30     17.90     -2.80        1.8   
3              3  400  강남  2017-04     14.60     30.30      3.80        1.8   
4              4  400  강남  2017-05     20.00     30.40     11.70        1.9   
...          ...  ...  ..      ...       ...       ...       ...        ...   
1461        1461    0  종로  2017-08     24.95     35.45     15.30        1.8   
1462        1462    0  종로  2017-09     21.10     29.40     11.40        1.7   
1463        1463    0  종로  2017-10     15.20     26.90      2.25        1.7   
1464        1464    0  종로  2017-11      5.05     17.65     -6.40        1.8   
1465        1465    0  종로  2017-12     -2.60      7.85    -13.20        1.9   

      월강수량합(mm)  
0         13.50  
1         12.50  
2         12.00  
3         57.00  
4         16.50  
...         ...  
1461     259.00  
1462      23.25  
1463      22.50  
1464      35.25  
1465      27.00  

[1466 rows x 9 columns]

In [14]:
# 기존에 가지고 있던 지역구별 화재발생건수를 한 열로 합쳐서,
# 전체 날씨 데이터에 붙여준다. 아래는 예시
df1 = pd.read_csv("강남구 전체 월별 화재발생건수.csv", encoding = 'cp949')
df2 = pd.read_csv("강동구 전체 월별 화재발생건수.csv", encoding = 'cp949')
pd.concat([df1,df2])

day  발생 건수
0   2017-01     56
1   2017-02     37
2   2017-03     41
3   2017-04     46
4   2017-05     53
..      ...    ...
55  2021-08     11
56  2021-09     18
57  2021-10     15
58  2021-11     18
59  2021-12     20

[120 rows x 2 columns]

In [20]:
# 날씨 데이터 순서대로 지역 이름 리스트를 다시 설정한다.
region_weather = ["강남구", "서초구", "강동구", "송파구", "강서구", "양천구",
                 "도봉구", "노원구", "동대문구", "중랑구", "동작구", "마포구", 
                 "서대문구", "광진구", "성북구", "용산구", "은평구", "금천구", 
                 "중구", "성동구", "구로구", "강북구", "관악구", "영등포구", 
                  "종로구"]

# 빈 데이터프레임 생성
merged_df = pd.DataFrame()

# 반복문 진행
for region in region_weather:
    df = pd.read_csv(f"{region} 전체 월별 화재발생건수.csv", encoding='cp949')
    merged_df = pd.concat([merged_df, df['발생 건수']])

# 데이터 정수형으로 바꿔주기
merged_df = merged_df.astype('int64')

# 결과 저장
merged_df.to_csv('모든 지역구 월별 화재발생건수.csv', encoding='cp949', index=False)

In [21]:
# 결과 확인
pd.read_csv("모든 지역구 월별 화재발생건수.csv", encoding = 'cp949')

0
0     56
1     37
2     41
3     46
4     53
...   ..
1495  13
1496  15
1497  14
1498  13
1499  21

[1500 rows x 1 columns]